In [2]:
import tensorflow as tf
import tensorflow.keras as kr
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
import pydicom as dicom
import multiprocessing as mp

folder_path = [os.path.join(r"/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data", i) for i in os.listdir(r"/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data")]
imgs = []

for i in folder_path:
    imgs.append(glob(i+"/*"))
print("Done")
folder_path = folder_path[1:]
imgs = imgs[1:]
print(imgs[0])
outputpath = []
inputpath = []
for j in imgs:
    for i in j:
        if "seg.nii.gz" in i.split("_"):
            outputpath.append(i)
for j in imgs:
    temppath = []
    for i in j:
        if "seg.nii.gz" not in i.split("_"):
            temppath.append(i)
    inputpath.append(temppath)
    
import tensorflow.keras as kr
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
       [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000 )])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
#load data
inputimg = np.load(r'/home/red/COMPDEEP/main/input_dataset.npy')
outputimg = np.load(r'/home/red/COMPDEEP/main/output_dataset.npy')

inputimg = np.array(inputimg, dtype=np.float32)
outputimg = np.array(outputimg, dtype=np.float32)
outputimg = np.expand_dims(outputimg, axis=3)

x_train = inputimg[:1126].copy()
y_train = outputimg[:1126].copy()
x_train.shape[0] == y_train.shape[0]
x_eval = inputimg[1126:1189].copy()
y_eval = outputimg[1126:1189].copy()
print(x_eval.shape, y_eval.shape)
print(x_eval.shape[0] == y_eval.shape[0])
x_test = inputimg[1189:].copy()
y_test = outputimg[1189:].copy()

print("train target:",y_train.shape,"                test target:",y_test.shape)
print(f"train x{x_test.shape}","                test x:", x_test.shape)

#shuffle data
np.random.seed(0)
np.random.shuffle(x_train)
np.random.shuffle(y_train)
np.random.shuffle(x_eval)
np.random.shuffle(y_eval)

np.random.shuffle(x_test)
np.random.shuffle(y_test)

y_train = y_train.astype(int)
y_eval = y_eval.astype(int)
y_test = y_test.astype(int)


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

input_shape = (240, 240, 4)
num_classes = 4

# Data preparation functions
#why my last code didnt work:
#the labels in the output goes 0,1,2,4????????
#yeah 0,1,2,4 not 0,1,2,3 but 0,1,2,4
#no wonder i am now depressed

#side note: np.unique(tells us all unique element)
def correct_labels(y):
    y[y == 4] = 3
    return y
#normalize data
def normalize_data(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Apply corrections to your data
x_train = normalize_data(x_train)
x_eval = normalize_data(x_eval)
x_test = normalize_data(x_test)

y_train = correct_labels(y_train)
y_eval = correct_labels(y_eval)
y_test = correct_labels(y_test)

# Rest of your code remains the same
def unet_model(input_shape, num_classes):
    inputs = keras.Input(input_shape)
    
    # Encoder (downsampling)
    conv1 = conv_block(inputs, 64)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = conv_block(pool1, 128)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = conv_block(pool2, 256)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Bridge
    conv4 = conv_block(pool3, 512)
    
    # Decoder (upsampling)
    up5 = layers.UpSampling2D(size=(2, 2))(conv4)
    up5 = layers.concatenate([up5, conv3])
    conv5 = conv_block(up5, 256)
    
    up6 = layers.UpSampling2D(size=(2, 2))(conv5)
    up6 = layers.concatenate([up6, conv2])
    conv6 = conv_block(up6, 128)
    
    up7 = layers.UpSampling2D(size=(2, 2))(conv6)
    up7 = layers.concatenate([up7, conv1])
    conv7 = conv_block(up7, 64)
    
    # Output layer
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(conv7)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def conv_block(input_tensor, num_filters):
    x = layers.Conv2D(num_filters, 3, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.1)(x)
    
    x = layers.Conv2D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

model = unet_model(input_shape, num_classes)

optimizer = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Print model summary
model.summary()

# Ensure your data is properly shaped
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

import tensorflow as tf

class_weights = {0: 1.4, 1: 4.0, 2: 2.0, 3: 5.0}  # Adjust these based on class frequencies



@tf.keras.utils.register_keras_serializable()
def weighted_sparse_categorical_crossentropy(y_true, y_pred):
    weights = tf.gather(list(class_weights.values()), tf.cast(y_true, tf.int32))
    unweighted_losses = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    weighted_losses = unweighted_losses * weights
    return tf.reduce_mean(weighted_losses)


y_train = np.squeeze(y_train, axis=-1)
y_eval = np.squeeze(y_eval, axis=-1)
y_test = np.squeeze(y_test, axis=-1)
model.compile(optimizer='adam', loss=weighted_sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(
    x_train, y_train,
    validation_data=(x_eval, y_eval),
    batch_size=16,
    epochs=80,
    callbacks=[
        keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5),
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)
with open('model_history.txt', 'w') as f:
    f.write(str(history.history))

print("done")




Done
['/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_flair.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_seg.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t1.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t1ce.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t2.nii.gz']
1 Physical GPUs, 1 Logical GPUs


2024-06-26 22:22:01.631470: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 22:22:01.816964: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 22:22:01.816999: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 22:22:01.822684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 22:22:01.822738: I external/local_xla/xla/stream_executor

(63, 240, 240, 4) (63, 240, 240, 1)
True
train target: (1126, 240, 240, 1)                 test target: (62, 240, 240, 1)
train x(62, 240, 240, 4)                 test x: (62, 240, 240, 4)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 240, 240,  │      2,368 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 240, 240,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 240, 240,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 240, 240,  │          0 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 240, 240,  │     36,928 │ dropout[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 240, 240,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 240, 240,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 120, 120,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 120, 120,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 120, 120,  │          0 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 120, 120,  │    147,584 │ dropout_1[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 60, 60,    │          0 │ activation_3[0][

...

 Trainable params: 7,789,316 (29.71 MB)

 Non-trainable params: 5,632 (22.00 KB)

x_train shape: (1126, 240, 240, 4)
y_train shape: (1126, 240, 240, 1)
Epoch 1/80


I0000 00:00:1719415340.949124    5962 service.cc:145] XLA service 0x7f3f10004000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719415340.950011    5962 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-06-26 22:22:21.268055: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1719415341.613517    5962 assert_op.cc:38] Ignoring Assert operator compile_loss/weighted_sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-06-26 22:22:21.973379: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-06-26 22:22:26.219604: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 241.00MiB with freed_by_count=0. The caller 

UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/red/.pyenv/versions/3.11.4/lib/python3.11/asyncio/events.py", lin

In [ ]:
#multiple class
y = x_test[0]
y = y.reshape(-1,240,240,4)
pred = model(y)[0]

def display_all_classes(pred):
    pred *=  127
    plt.figure(figsize=(20, 5))
    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.imshow(pred[:,:,i], cmap='bone')
        plt.title(f"Class {i}")
        plt.colorbar()
        plt.axis('off')
    plt.show()
display_all_classes(pred)




In [ ]:
#display truth mask
y = x_eval[0]
y = y.reshape(-1, 240, 240, 4)
pred = model.predict(y)[0]
def display_mask(pred):
    mask = np.argmax(pred, axis=-1)
    print("Mask shape:", mask.shape)
    print("Unique values in mask:", np.unique(mask))
    print("Mask min:", np.min(mask))
    print("Mask max:", np.max(mask))
    
    plt.figure(figsize=(10, 10))
    plt.imshow(mask,cmap = "bone")
    plt.colorbar()
    plt.title("Segmentation Mask")
    plt.axis('off')
    plt.show()

display_mask(pred)

In [ ]:
#display mask of prediction
y = x_test[0]
y = y.reshape(-1, 240, 240, 4)
pred = model(y)[0]

import numpy as np
import matplotlib.pyplot as plt

def display_segmentation(pred):
    # Convert probabilities to class labels
    segmentation = np.argmax(pred, axis=-1)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(segmentation, cmap='nipy_spectral')
    plt.colorbar(ticks=range(4), label='Class')
    plt.title("Segmentation Prediction")
    plt.axis('off')
    plt.show()

# Predict and display
y = x_test[0]
y = y.reshape(-1, 240, 240, 4)
pred = model.predict(y)[0]  # Use model.predict instead of model(y)
display_segmentation(pred)


print("Prediction shape:", pred.shape)
print("Prediction min:", np.min(pred))
print("Prediction max:", np.max(pred))
print("Unique values in prediction:", np.unique(pred))